In [1]:
%pylab inline
# Pandas package... using data frames in python
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from numpy import genfromtxt, savetxt
import random

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('C:\\Users\\anirudhbedre\\Desktop\\SF Crime\\train.csv', header=0)
#type(df)
#df.describe() 
#df.dtypes 
#df.info()
#df.describe()
df.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

In [3]:
#get dummy variables for day of the week
dummies = pd.get_dummies(df['DayOfWeek'])
df = df.join(dummies)
#get dummy variables for district
dummies2 = pd.get_dummies(df['PdDistrict'])
df = df.join(dummies2)
#convert to datetime format
df['Dates'] = pd.to_datetime(df['Dates'])
#create and get 'hour' column from 'Dates'
df['hour'] = df['Dates'].map( lambda x: pd.to_datetime(x).hour )
#get dummy variables for hour
dummies3 = pd.get_dummies(df['hour'])
df = df.join(dummies3)
#drop excess dummy variables
df= df.drop([0,'Monday','BAYVIEW'],axis=1)
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 878049 entries, 0 to 878048
Data columns (total 48 columns):
Dates         878049 non-null datetime64[ns]
Category      878049 non-null object
Descript      878049 non-null object
DayOfWeek     878049 non-null object
PdDistrict    878049 non-null object
Resolution    878049 non-null object
Address       878049 non-null object
X             878049 non-null float64
Y             878049 non-null float64
Friday        878049 non-null float64
Saturday      878049 non-null float64
Sunday        878049 non-null float64
Thursday      878049 non-null float64
Tuesday       878049 non-null float64
Wednesday     878049 non-null float64
CENTRAL       878049 non-null float64
INGLESIDE     878049 non-null float64
MISSION       878049 non-null float64
NORTHERN      878049 non-null float64
PARK          878049 non-null float64
RICHMOND      878049 non-null float64
SOUTHERN      878049 non-null float64
TARAVAL       878049 non-null float64
TENDERLOIN    

In [4]:
cols = df.columns.tolist()
#rearranging columns  -- shifted 'hour' column in front here
cols = ['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'hour', 'Friday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 1L, 2L, 3L, 4L, 5L, 6L, 7L, 8L, 9L, 10L, 11L, 12L, 13L, 14L, 15L, 16L, 17L, 18L, 19L, 20L, 21L, 22L, 23L]
df = df[cols]

In [5]:
# Split train and validation data
check = pd.DataFrame(np.random.randn(878049))   #generate random nos. size=(no.of records)
rows= random.sample(check.index, 526829)
# 878049 rec  60%= 526829 40%= 351220
len(rows)

526829

In [6]:

df_60=df.ix[rows]
len(df_60)


526829

In [7]:
df_40=df.drop(rows)
len(df_40)

351220

In [8]:
df_60 = df_60.drop(['Dates', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'hour'],axis=1)
df_60.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526829 entries, 92293 to 285116
Data columns (total 39 columns):
Category      526829 non-null object
Friday        526829 non-null float64
Saturday      526829 non-null float64
Sunday        526829 non-null float64
Thursday      526829 non-null float64
Tuesday       526829 non-null float64
Wednesday     526829 non-null float64
CENTRAL       526829 non-null float64
INGLESIDE     526829 non-null float64
MISSION       526829 non-null float64
NORTHERN      526829 non-null float64
PARK          526829 non-null float64
RICHMOND      526829 non-null float64
SOUTHERN      526829 non-null float64
TARAVAL       526829 non-null float64
TENDERLOIN    526829 non-null float64
1             526829 non-null float64
2             526829 non-null float64
3             526829 non-null float64
4             526829 non-null float64
5             526829 non-null float64
6             526829 non-null float64
7             526829 non-null float64
8           

In [9]:
df_40 = df_40.drop(['Dates', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'hour'],axis=1)
df_40.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351220 entries, 0 to 878044
Data columns (total 39 columns):
Category      351220 non-null object
Friday        351220 non-null float64
Saturday      351220 non-null float64
Sunday        351220 non-null float64
Thursday      351220 non-null float64
Tuesday       351220 non-null float64
Wednesday     351220 non-null float64
CENTRAL       351220 non-null float64
INGLESIDE     351220 non-null float64
MISSION       351220 non-null float64
NORTHERN      351220 non-null float64
PARK          351220 non-null float64
RICHMOND      351220 non-null float64
SOUTHERN      351220 non-null float64
TARAVAL       351220 non-null float64
TENDERLOIN    351220 non-null float64
1             351220 non-null float64
2             351220 non-null float64
3             351220 non-null float64
4             351220 non-null float64
5             351220 non-null float64
6             351220 non-null float64
7             351220 non-null float64
8             35

In [10]:
target=df_60['Category']
target

92293             LARCENY/THEFT
636577            DRUG/NARCOTIC
275953             NON-CRIMINAL
581364            LARCENY/THEFT
558542            VEHICLE THEFT
255457                 BURGLARY
358986           OTHER OFFENSES
629587                 WARRANTS
876206                  ASSAULT
512403                  ROBBERY
233816                  ROBBERY
525998           OTHER OFFENSES
242077    SEX OFFENSES FORCIBLE
553033          FAMILY OFFENSES
729756             NON-CRIMINAL
849384                  ROBBERY
747713            DRUG/NARCOTIC
366434           SUSPICIOUS OCC
816873             NON-CRIMINAL
610392                  ASSAULT
279294            LARCENY/THEFT
683221                VANDALISM
314782                  ROBBERY
572815                VANDALISM
30798              NON-CRIMINAL
614719                 WARRANTS
654279           SUSPICIOUS OCC
533094           SUSPICIOUS OCC
244767           OTHER OFFENSES
835236            LARCENY/THEFT
                  ...          
680513  

In [11]:
train = df_60.iloc[:,1:]
lr = LogisticRegression()
lr.fit(train, target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [12]:
train_results=lr.predict_proba(train)

In [14]:
duumies_train=pd.get_dummies(df_60.iloc[:,0])
from sklearn.metrics import log_loss

log_loss(duumies_train, train_results)  

2.5772394998070398

In [15]:
valid = df_40.iloc[:,1:]
dummies_valid=pd.get_dummies(df_40.iloc[:,0])
dummies_valid

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
valid_results=lr.predict_proba(valid)

In [17]:
log_loss(dummies_valid, valid_results)  

2.5814145482582225

In [63]:
# savetxt('sf_valid_results.csv', valid_results, delimiter=',', fmt='%f')
#savetxt('sf_dummies_valid.csv', dummies_valid, delimiter=',', fmt='%f')

In [18]:
# train whole data set
df_total = df.drop(['Dates', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'hour'],axis=1)
df_total

,Category,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday,CENTRAL,INGLESIDE,MISSION,...,14,15,16,17,18,19,20,21,22,23
0,WARRANTS,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,OTHER OFFENSES,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,OTHER OFFENSES,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,LARCENY/THEFT,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,LARCENY/THEFT,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,LARCENY/THEFT,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6,VEHICLE THEFT,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
7,VEHICLE THEFT,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,LARCENY/THEFT,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,LARCENY/THEFT,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:

total_train = df_total.iloc[:,1:]
total_target= df_total['Category']
lr = LogisticRegression()
lr.fit(total_train,total_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [21]:
valid_results2=lr.predict_proba(valid)

In [22]:
log_loss(dummies_valid, valid_results2) 

2.5792422668971517

In [23]:
td = pd.read_csv('C:\\Users\\anirudhbedre\\Desktop\\SF Crime\\test.csv', header=0)

In [24]:
#get dummy variables for day of the week
dummies = pd.get_dummies(td['DayOfWeek'])
td = td.join(dummies)
#get dummy variables for district
dummies2 = pd.get_dummies(td['PdDistrict'])
td = td.join(dummies2)
#convert to datetime format
td['Dates'] = pd.to_datetime(td['Dates'])
#create and get 'hour' column from 'Dates'
td['hour'] = td['Dates'].map( lambda x: pd.to_datetime(x).hour )
#get dummy variables for hour
dummies3 = pd.get_dummies(td['hour'])
td = td.join(dummies3)

#drop excess dummy variables
td= td.drop([0,'Monday','BAYVIEW'],axis=1)
td.info()
cols = df.columns.tolist()
cols = ['Id','Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'hour', 'Friday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 1L, 2L, 3L, 4L, 5L, 6L, 7L, 8L, 9L, 10L, 11L, 12L, 13L, 14L, 15L, 16L, 17L, 18L, 19L, 20L, 21L, 22L, 23L]
td = td.drop(['Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'hour'],axis=1)
td_ids= td['Id']
td=td.drop(['Id'],axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884262 entries, 0 to 884261
Data columns (total 46 columns):
Id            884262 non-null int64
Dates         884262 non-null datetime64[ns]
DayOfWeek     884262 non-null object
PdDistrict    884262 non-null object
Address       884262 non-null object
X             884262 non-null float64
Y             884262 non-null float64
Friday        884262 non-null float64
Saturday      884262 non-null float64
Sunday        884262 non-null float64
Thursday      884262 non-null float64
Tuesday       884262 non-null float64
Wednesday     884262 non-null float64
CENTRAL       884262 non-null float64
INGLESIDE     884262 non-null float64
MISSION       884262 non-null float64
NORTHERN      884262 non-null float64
PARK          884262 non-null float64
RICHMOND      884262 non-null float64
SOUTHERN      884262 non-null float64
TARAVAL       884262 non-null float64
TENDERLOIN    884262 non-null float64
hour          884262 non-null int64
1             8

In [25]:
test_results=lr.predict_proba(td)

In [26]:
test_results=pd.DataFrame(test_results)
test_results.to_csv(path_or_buf='sf_tmp_results8.csv')